# Conversion of forcing created for MESH to HYPE nc or txt format for model simulation.
## nc creation is based on this page:
http://www.smhi.net/hype/wiki/doku.php?id=start:hype_file_reference:nnobs.nc

In [1]:
# packages are loaded
import xarray as xr
import pint_xarray
import glob
import netCDF4 as nc4
import os
import sys
import pandas as pd
from   easymore import Utility as esmrut
import subprocess
import pint
_ureg = pint.UnitRegistry(force_ndarray_like=True)
_ureg.define('millibar = 1e-3 * bar')

In [2]:
# inputs
# Set the folder path where the remapped .nc file is located for MESH (it can be any remapped nc file)
nc_file_source = '/home/shg096/scratch/test/easymore_outputs/remapped_ERA5/*.nc'
path_to_save = '/home/shg096/scratch/test/HYPE/'

if not os.path.isdir(path_to_save):
    os.makedirs(path_to_save)

In [3]:
# step 1 - merge the remapped files
outfile = path_to_save+'outfile.nc'
command = 'load cdo; cdo mergetime '+nc_file_source+' '+outfile
subprocess.run(command, shell=True)
# load ds
ds = xr.open_dataset(outfile)

/bin/sh: load: command not found

cdo    mergetime (Abort): Outputfile /home/shg096/scratch/test/HYPE/outfile.nc already exists!


In [7]:
# Drop all variables except needed variables
ds_Pobs = ds.drop_vars([var for var in ds.variables if var not in ['time', 'COMID', 'latitude', 'longitude', 'pptrate']])


# aggregate from hourly to daily
ds_Pobs  = esmrut.agg_hourl_to_daily (ds_Pobs, offset=3, stat='mean')

# change the unit for the target variables
ds_Pobs  = esmrut.convert_units(ds_Pobs,
                                units = {'pptrate':'mm s**-1'},
                                unit_registry = _ureg,
                                to_units= {'pptrate':'mm day**-1'})

# rename the variable
ds_Pobs = ds_Pobs.rename({'pptrate': 'Pobs'})
ds_Pobs = ds_Pobs.rename({'COMID': 'id'})
ds_Pobs.to_netcdf(path_to_save+'Pobs.nc')

df_Pobs = pd.DataFrame(ds_Pobs['Pobs'].values, index=ds_Pobs['time'], columns=ds_Pobs['id'])
df_Pobs.index.name = 'time'
df_Pobs.to_csv(path_to_save+'Pobs.cav', sep='\t', na_rep='', index_label='time', float_format='%.3f')


In [10]:
# Drop all variables except needed variables
ds_TMAXobs = ds.drop_vars([var for var in ds.variables if var not in ['time', 'COMID', 'latitude', 'longitude', 'airtemp']])


# aggregate from hourly to daily
ds_TMAXobs  = esmrut.agg_hourl_to_daily (ds_TMAXobs, offset=3, stat='max')

# change the unit for the target variables
ds_TMAXobs  = esmrut.convert_units(ds_TMAXobs,
                                   units = {'airtemp':'kelvin'},
                                   unit_registry = _ureg,
                                   to_units= {'airtemp':'degC'})

# rename the variable
ds_TMAXobs = ds_TMAXobs.rename({'airtemp': 'TMAXobs'})
ds_TMAXobs = ds_TMAXobs.rename({'COMID': 'id'})
ds_TMAXobs.to_netcdf(path_to_save+'TMAXobs.nc')

# save the values
df_TMAXobs = pd.DataFrame(ds_TMAXobs['TMAXobs'].values, index=ds_TMAXobs['time'], columns=ds_TMAXobs['id'])
df_TMAXobs.index.name = 'time'
df_TMAXobs.to_csv(path_to_save+'TMAXobs.cav', sep='\t', na_rep='', index_label='time', float_format='%.3f')


In [11]:
# Drop all variables except needed variables
ds_TMINobs = ds.drop_vars([var for var in ds.variables if var not in ['time', 'COMID', 'latitude', 'longitude', 'airtemp']])


# aggregate from hourly to daily
ds_TMINobs  = esmrut.agg_hourl_to_daily (ds_TMINobs, offset=3, stat='min')

# change the unit for the target variables
ds_TMINobs  = esmrut.convert_units(ds_TMINobs,
                                   units = {'airtemp':'kelvin'},
                                   unit_registry = _ureg,
                                   to_units= {'airtemp':'degC'})

# rename the variable and save
ds_TMINobs = ds_TMINobs.rename({'airtemp': 'TMINobs'})
ds_TMINobs = ds_TMINobs.rename({'COMID': 'id'})
ds_TMINobs.to_netcdf(path_to_save+'TMINobs.nc')

# save the dataframe for no index version
df_TMINobs = pd.DataFrame(ds_TMINobs['TMINobs'].values, index=ds_TMINobs['time'], columns=ds_TMINobs['id'])
df_TMINobs.index.name = 'time'
df_TMINobs.to_csv(path_to_save+'TMINobs.cav', sep='\t', na_rep='', index_label='time', float_format='%.3f')
